In [265]:
import json
import re
from collections import Counter
import inflect
import random

In [360]:
random.seed(42)

all_train = json.load(open('raw_data/all_train.json',encoding='utf-8'))
test = json.load(open('raw_data/test_set_new.json',encoding='utf-8'))
all = all_train + test
no_task = [x for x in all if x.get('task_name', None) == None]
all = [x for x in all if x.get('task_name', None) != None]

def prepare_dataset(dataset):
    sign_dict = {'red': 'negative', 'green': 'positive', 'yellow': 'negligible'}

    for i in range(len(dataset)):
        # Some of the data is in string form, eval() is to convert it to dict
        try:
            dataset[i]['feature_division'] = eval(dataset[i]['feature_division'])
        except:
            dataset[i]['feature_division'] = dataset[i]['feature_division']
        dataset[i]['feature_division']['explainable_df'] = eval(dataset[i]['feature_division']['explainable_df'])
        
        # Some of the fields we want are inside the feature_division dict, moving them to the top level
        dataset[i]['values'] = [format(val, '.2f') for val in dataset[i]['feature_division']['explainable_df']['Values'].values()]
        ft_nums =[re.search('F\d*', val).group() for val in list(dataset[i]['feature_division']['explainable_df']['annotate_placeholder'].values())]
        ft_names = list(dataset[i]['feature_division']['explainable_df']['Variable'].values())
        dataset[i]['sign'] = [sign_dict[x] for x in dataset[i]['feature_division']['explainable_df']['Sign'].values()]
        dataset[i]['narrative_id'] = dataset[i].pop('id')
        dataset[i]['unique_id'] = i
        dataset[i]['classes_dict'] = {v[0].strip(): v[1].strip() for v in [y.split(':') for y in [x for x in dataset[i]['prediction_confidence_level'].split(',')]]}
        dataset[i]['narrative_questions'] = dataset[i]['narrative_question'].strip('<ul><li>/ ').split(' </li> <li> ')
        
        # Shuffle feature names to ensure separation of train and test
        new_ft_nums = ft_nums.copy()
        random.shuffle(new_ft_nums)
        old2new_ft_nums = dict(zip(ft_nums, new_ft_nums))
        ft_ptn = re.compile("|".join([f'{k}\\b' for k in old2new_ft_nums.keys()]))
        
        dataset[i]['feature_nums'] = new_ft_nums
        dataset[i]['ft_num_to_name'] = dict(zip(new_ft_nums, ft_names))
        dataset[i]['old2new_ft_nums'] = old2new_ft_nums
        dataset[i]['narration'] = ft_ptn.sub(lambda m: old2new_ft_nums[re.escape(m.group(0))], dataset[i]['narration'])
        dataset[i]['narrative_questions'] = [ft_ptn.sub(lambda m: old2new_ft_nums[re.escape(m.group(0))], x) for x in dataset[i]['narrative_questions']]
        
        # # Shuffle class names too
        new_classes = list(dataset[i]['classes_dict'].keys()).copy()
        random.shuffle(new_classes)
        old2new_classes = dict(zip(list(dataset[i]['classes_dict'].keys()), new_classes))
        cls_ptn = re.compile("|".join([f'{k}\\b' for k in old2new_classes.keys()]))
        
        dataset[i]['predicted_class'] = cls_ptn.sub(lambda m: old2new_classes[re.escape(m.group(0))], dataset[i]['predicted_class'])
        dataset[i]['narration'] = cls_ptn.sub(lambda m: old2new_classes[re.escape(m.group(0))], dataset[i]['narration'])
        dataset[i]['classes_dict'] = {old2new_classes[k]: v for k, v in dataset[i]['classes_dict'].items()}
        dataset[i]['narrative_questions'] = [cls_ptn.sub(lambda m: old2new_classes[re.escape(m.group(0))], x) for x in dataset[i]['narrative_questions']]
        dataset[i]['old2new_classes'] = old2new_classes

        
        for key in ['deleted', 'mturk_id','narrative_status', 'predicted_class_label', 'date_submitted',
                    'date_approved', 'features_placeholder', 'is_paid', 'redeem_code', 'narrator',
                    'user_ip','feature_division', 'narrative_question', 'prediction_confidence_level']:
            dataset[i].pop(key)
        return dataset
all = prepare_dataset(all)
# no_task = prepare_dataset(no_task)
json.dump(all, open('jb_data/all.json', 'w', encoding='utf-8'), indent=4)

# Split into train, test, val 80:10:10
random.shuffle(all)
train = all[:int(0.8*len(all))]
test = all[int(0.8*len(all)):int(0.9*len(all))]
val = all[int(0.9*len(all)):]

json.dump(train, open('jb_data/train.json', 'w', encoding='utf-8'), indent=4)
json.dump(test, open('jb_data/test.json', 'w', encoding='utf-8'), indent=4)
json.dump(val, open('jb_data/val.json', 'w', encoding='utf-8'), indent=4)


In [335]:
new_ft_nums = ft_nums.copy()
random.shuffle(new_ft_nums)
old2new_ft_nums = dict(zip(ft_nums, new_ft_nums))
ft_ptn = re.compile("|".join([f'{k}\\b' for k in old2new_ft_nums.keys()]))
ft_ptn.sub(lambda m: old2new_ft_nums[re.escape(m.group(0))], narration)

'In summary, the model predicted an 87.14% likelihood of the class label C1 for the test example under consideration, therefore, there is a chance of about 12.86% that the correct class label could be a different label. The features with the highest impact on the model are F23, F24, F18, and F9, whose values are attributing most to the labeling decision here and among these features, only F9 shows the potential to shift the narrative toward a different label. On impact comparison, features F23, F24, F18 and F9 have higher impact on the model prediction than F17 and F6. Features F23, F24, F18, F17, and F6 show a positive impact shifting towards the prediction of C1. F9 is the most negative of all the set of features passed to the model, F1, F25, and F13 have moderate negative influence, whereas the feature F8 has very little negative impact on the prediction.'

In [336]:
narration

'In summary, the model predicted an 87.14% likelihood of the class label C1 for the test example under consideration, therefore, there is a chance of about 12.86% that the correct class label could be a different label. The features with the highest impact on the model are F1, F2, F3, and F11, whose values are attributing most to the labeling decision here and among these features, only F11 shows the potential to shift the narrative toward a different label. On impact comparison, features F1, F2, F3 and F11 have higher impact on the model prediction than F12 and F6. Features F1, F2, F3, F12, and F6 show a positive impact shifting towards the prediction of C1. F11 is the most negative of all the set of features passed to the model, F4, F7, and F10 have moderate negative influence, whereas the feature F8 has very little negative impact on the prediction.'

In [337]:
old2new_ft_nums

{'F8': 'F8',
 'F2': 'F24',
 'F1': 'F23',
 'F21': 'F20',
 'F25': 'F15',
 'F10': 'F13',
 'F3': 'F18',
 'F9': 'F10',
 'F15': 'F11',
 'F7': 'F25',
 'F20': 'F16',
 'F12': 'F17',
 'F24': 'F21',
 'F6': 'F6',
 'F17': 'F26',
 'F23': 'F5',
 'F11': 'F9',
 'F22': 'F2',
 'F4': 'F1',
 'F14': 'F3',
 'F19': 'F4',
 'F18': 'F12',
 'F16': 'F14',
 'F13': 'F19',
 'F5': 'F7',
 'F26': 'F22'}

# Linearisation

In [268]:
'| predicted class | C4 100.00% | other classes | C2 0.00%; C3 0.00%; C1 0.00% |'


other_classes = "&& ".join([f"{k} {v}" for k,v in dataset[0]['classes_dict'].items() if k != chosen_class])
just_fts = ' '.join(dataset[0]['feature_nums'])
fts_and_signs = "&& ".join([f'{a} {b} ' for a, b in zip(dataset[0]['feature_nums'], dataset[0]['sign'])])
fts_and_pos = "&& ".join([f'{a} {b} ' for a, b in zip(dataset[0]['feature_nums'], dataset[0]['sign']) if b == 'positive'])
fts_and_nega = "&& ".join([f'{a} {b} ' for a, b in zip(dataset[0]['feature_nums'], dataset[0]['sign']) if b == 'negative'])
fts_and_negl = "&& ".join([f'{a} {b} ' for a, b in zip(dataset[0]['feature_nums'], dataset[0]['sign']) if b == 'negligible'])
fts_and_negl = 'None' if fts_and_negl == '' else fts_and_negl

essel_input = f'| predicted class | {chosen_class} {dataset[0]["classes_dict"][chosen_class]} | other classes | {other_classes} | \
features | {fts_and_signs}| postive features | {fts_and_pos} | negative features | {fts_and_nega} | \
negligible features | {fts_and_negl} |'

p = inflect.engine()

ordinals = [p.ordinal(i+1) for i in range(len(dataset[0]['feature_nums']))]

features = ' '.join([f'| {o} | {f} {s} {v}' for o, f, s, v in zip(ordinals, dataset[0]['feature_nums'], dataset[0]['sign'], dataset[0]['values'])])
    
ord_first_input = f'| predicted class | {chosen_class} {dataset[0]["classes_dict"][chosen_class]} | other classes | {other_classes} {features} |'

ft_first_input = ' '.join([f'| {f} | {o} {s} {v}' for o, f, s, v in zip(ordinals, dataset[0]['feature_nums'], dataset[0]['sign'], dataset[0]['values'])])



| predicted class | C1 91.36% | other classes | C2 8.64% | features | F3 negative && F4 negative && F6 positive && F12 positive && F11 positive && F14 positive && F13 negative && F9 positive && F10 negative && F15 positive && F5 positive && F7 positive && F2 positive && F8 negative && F1 positive | postive features | F6 positive && F12 positive && F11 positive && F14 positive && F9 positive && F15 positive && F5 positive && F7 positive && F2 positive && F1 positive  | negative features | F3 negative && F4 negative && F13 negative && F10 negative && F8 negative  | negligible features | None |
| predicted class | C1 91.36% | other classes | C2 8.64% | 1st | F3 negative -0.30 | 2nd | F4 negative -0.25 | 3rd | F6 positive 0.23 | 4th | F12 positive 0.15 | 5th | F11 positive 0.09 | 6th | F14 positive 0.09 | 7th | F13 negative -0.07 | 8th | F9 positive 0.07 | 9th | F10 negative -0.06 | 10th | F15 positive 0.05 | 11th | F5 positive 0.05 | 12th | F7 positive 0.02 | 13th | F2 positive 0.02 | 14t

Essels way is like

'prediction: predictionlabel && predictionlabel: 69.02% && predictionrankB: 30.98% <|section-sep|> featAP featCN featDP featEN featFN featGP featHP featIP featJP featKP <|section-sep|> <mentions> featAP featDP featGP featHP featIP featJP featKP <|section-sep|> featCN featEN featFN <|section-sep|> </mentions> <|section-sep|> <explain>'

In [276]:


dataset[0]['narrative_question'].strip('<ul><li>/ ').split(' </li> <li> ')

['Provide a statement summarizing the prediction made for the test case.',
 'For the current test instance, describe the direction of influence of the following features: F3 (value equal to  V0) and F4 (with a value equal to  V0).',
 'Compare and contrast the impact of the following features  (F6, F12, F11 and F14) on the model’s prediction of C1.',
 'Describe the degree of impact of the following features: F13, F9, F10 and F15?']

In [266]:
' '.join([f'| {f} | {o} {s} {v}' for o, f, s, v in zip(ordinals, dataset[0]['feature_nums'], dataset[0]['sign'], dataset[0]['values'])])

'| F3 | 1st negative -0.30 | F4 | 2nd negative -0.25 | F6 | 3rd positive 0.23 | F12 | 4th positive 0.15 | F11 | 5th positive 0.09 | F14 | 6th positive 0.09 | F13 | 7th negative -0.07 | F9 | 8th positive 0.07 | F10 | 9th negative -0.06 | F15 | 10th positive 0.05 | F5 | 11th positive 0.05 | F7 | 12th positive 0.02 | F2 | 13th positive 0.02 | F8 | 14th negative -0.01 | F1 | 15th positive 0.01'

In [238]:
"&& ".join([f'{a} {b} ' for a, b in zip(dataset[0]['feature_nums'], dataset[0]['sign']) if b == 'positive'])

'F6 positive && F12 positive && F11 positive && F14 positive && F9 positive && F15 positive && F5 positive && F7 positive && F2 positive && F1 positive '

In [214]:
classes_dict = {'C1': '20.00%', 'C2': '80.00%', 'C3': '0.00%', 'C4': '0.00%'}

1st
6th
11th
16th
21st
